In [1]:
from random import seed
from csv import reader
from math import exp
from random import randrange
from random import random
prec = []
rec = []
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)     
    return dataset[1:]

In [2]:
def str_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

def str_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    uniqueName = set(class_values)
    traverse = dict()
    for i, value in enumerate(uniqueName):
        traverse[value] = i
    for row in dataset:
        row[column] = traverse[row[column]]
    return traverse

In [3]:
def datasetMinmax(dataset):
    minmax = list()
    statistics = [[min(column), max(column)] for column in zip(*dataset)]
    return statistics

def normalizeDataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)-1):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])


In [4]:
def cross_validation_split(dataset, n_folds):
    datasetSplit = list()
    datasetCopy = list(dataset)
    foldSize = int(len(dataset) / n_folds)
    for i in range(n_folds):
        folds = list()
        while len(folds) < foldSize:
            index = randrange(len(datasetCopy))
            folds.append(datasetCopy.pop(index))
        datasetSplit.append(folds)
    return datasetSplit


In [16]:
def accuracy(actual, predicted):
    true = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            true += 1
        true_positive = 0
        false_positive = 0
        true_negative = 0
        false_negative = 0
        if actual[i] == 1 and predicted[i] == 1:
            true_positive += 1
        if actual[i] == 1 and predicted[i] == 0:
            false_negative += 1
        if actual[i] == 0 and predicted[i] == 1:
            false_positive += 1
        if actual[i] == 0 and predicted[i] == 0:
            true_negative += 1
    prec.append([true_positive, true_negative, false_positive, false_negative])
    return true / float(len(actual)) * 100.0


In [17]:
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accur = accuracy(actual, predicted)
        scores.append(accur)
    return scores

In [18]:
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

def transfer_derivative(output):
    return output * (1.0 - output)

def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)

def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights': [random() for i in range(n_inputs + 1)]}
                    for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights': [random() for i in range(n_hidden + 1)]}
                    for i in range(n_outputs)]
    network.append(output_layer)
    return network

def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

def back_propagation(train, test, l_rate, n_epoch, n_hidden):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs)
    predictions = list()
    for row in test:
        prediction = predict(network, row)
        predictions.append(prediction)
    return(predictions)

In [19]:
def main():
    seed(1)

    filename = 'Datasets/heart.csv'
    dataset = load_csv(filename)
    for i in range(len(dataset[0])-1):
        str_to_float(dataset, i)
    str_to_int(dataset, len(dataset[0])-1)

    minmax = datasetMinmax(dataset)
    normalizeDataset(dataset, minmax)

    n_folds = 10
    l_rate = 0.2
    n_epoch = 500
    n_hidden = 5
    scores = evaluate_algorithm( dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
    print('Scores:')
    for i in range(len(scores)):
        print(i+1, ": ", scores[i])
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
    ttp = 0
    ttn = 0
    tfp = 0
    tfn = 0
    for i in range(len(prec)):
        ttp += prec[i][0]
        ttn += prec[i][1]
        tfp += prec[i][2]
        tfn += prec[i][3]

    print('Precision: ', ttp / (ttp + tfp))
    print('Recall: ', ttp / (ttp + tfn))
if __name__ == '__main__':
    main()

Scores:
1 :  86.66666666666667
2 :  80.0
3 :  83.33333333333334
4 :  90.0
5 :  83.33333333333334
6 :  70.0
7 :  80.0
8 :  76.66666666666667
9 :  86.66666666666667
10 :  83.33333333333334
Mean Accuracy: 82.000%
Precision:  0.4
Recall:  0.6666666666666666
